In [1]:
import os

In [2]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Virtuoso633"


In [6]:
import dagshub
dagshub.init(repo_owner='Virtuoso633', repo_name='Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Virtuoso633

Initialized MLflow to track repo "Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2"

Repository Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2 initialized!

2024/10/15 08:45:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run intelligent-quail-22 at: https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow/#/experiments/0/runs/7a9884b25aea4143b77fc6c6433cc87c.
2024/10/15 08:45:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow/#/experiments/0.


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from MlOpsProject.constants import *
from MlOpsProject.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.Target_Column

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_url="https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow",
        
        )

        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-15 08:46:16,896: INFO: common: YAML file: config/config.yaml loaded successfully.]
[2024-10-15 08:46:16,900: INFO: common: YAML file: params.yaml loaded successfully.]
[2024-10-15 08:46:16,903: INFO: common: YAML file: schema.yaml loaded successfully.]
[2024-10-15 08:46:16,904: INFO: common: created directory at: artifacts]
[2024-10-15 08:46:16,904: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-15 08:46:17,617: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/10/15 08:46:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.
2024/10/15 08:46:28 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/10/15 08:46:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-mole-921 at: https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow/#/experiments/0/runs/5747e6c5a7bf4e9db40b325a2d221272.
2024/10/15 08:46:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Virtuoso633/Full-Stack-Machine-Learning-Deployment-with-MLOps-and-AWS-EC2.mlflow/#/experiments/0.
